In [2]:
from roboflow import Roboflow

rf = Roboflow(api_key='2OXz0BvJfNWjz1w7X5Hg')

workspace = rf.workspace("roboflow-jvuqo")
project = workspace.project("football-field-detection-f07vi")
version = project.version(14)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...


In [ ]:
from ultralytics import YOLO
import os.path

# Create a new YOLO model from scratch
#model = YOLO("yolov8x-pose.yaml")

# Load a pretrained YOLO model
model = YOLO("yolov8x-pose.pt")
path=os.path.join(dataset.location,'data.yaml')
# Train the model using the dataset for 500 epochs
results = model.train(task='pose', data=path, batch=5, epochs=10, imgsz=640, mosaic=0.0)

Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.1+cu124 CPU (Intel Core(TM) Ultra 7 155H)
engine/trainer: task=pose, mode=train, model=yolov8x-pose.pt, data=/home/lucasburgessdev/code/lucasburgessdev/sports_camera_calibration/football-field-detection-14/data.yaml, epochs=1, time=None, patience=100, batch=1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_

train: Scanning /home/lucasburgessdev/code/lucasburgessdev/sports_camera_calibration/football-field-detection-14/train/labels.cache... 255 images, 0 backgrounds, 0 corrupt: 100%|██████████| 255/255 [00:00<?, ?it/s]
val: Scanning /home/lucasburgessdev/code/lucasburgessdev/sports_camera_calibration/football-field-detection-14/valid/labels.cache... 34 images, 0 backgrounds, 0 corrupt: 100%|██████████| 34/34 [00:00<?, ?it/s]

Plotting labels to runs/pose/train11/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 103 weight(decay=0.0), 113 weight(decay=0.0005), 112 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/pose/train11
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G     0.9086      11.52     0.6873     0.8034      1.662          1        640: 100%|██████████| 255/255 [19:03<00:00,  4.48s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:50<00:00,  2.96s/it]


                   all         34         34      0.236      0.912      0.384      0.157          0          0          0          0

1 epochs completed in 0.333 hours.
Optimizer stripped from runs/pose/train11/weights/last.pt, 140.0MB
Optimizer stripped from runs/pose/train11/weights/best.pt, 140.0MB

Validating runs/pose/train11/weights/best.pt...
Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.1+cu124 CPU (Intel Core(TM) Ultra 7 155H)
YOLOv8x-pose summary (fused): 287 layers, 69,784,275 parameters, 0 gradients, 264.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:46<00:00,  2.76s/it]


                   all         34         34      0.236      0.912      0.384      0.157          0          0          0          0
Speed: 1.2ms preprocess, 1370.2ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/pose/train11


In [ ]:
import supervision as sv
from inference import get_model
import cv2
import numpy as np
from IPython.display import Image


model = get_model(
    model_id="football-field-detection-f07vi/14", 
    api_key='2OXz0BvJfNWjz1w7X5Hg'
)

path='/home/lucasburgessdev/code/lucasburgessdev/sports_camera_calibration/football-field-detection-14/test/images/121364_3_7_png.rf.94ce64904559a871c7b28c995b425022.jpg'
image = cv2.imread(path)
result = model.infer(image, confidence=0.3)[0]

keypoints = sv.KeyPoints.from_inference(result)

filter = keypoints.confidence > 0.5
keypoints.xy = keypoints.xy[filter][np.newaxis]
keypoints.confidence = keypoints.confidence[filter][np.newaxis]

vertex_annotator = sv.VertexAnnotator(radius=12)
annotated_image = vertex_annotator.annotate(image.copy(), keypoints)
#print(annotated_image)
Image(annotated_image)
